# 🎨 Neural Style Transfer with VGG19 (Google Colab Ready)

This notebook uses TensorFlow and a pre-trained VGG19 model to apply the artistic style of one image (e.g., Van Gogh's *Starry Night*) to the content of another image (e.g., Golden Gate Bridge photo).

In [ ]:
# ✅ STEP 1: Install required libraries
!pip install tensorflow matplotlib --quiet

In [ ]:
# ✅ STEP 2: Import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import time
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing import image as kp_image

In [ ]:
# ✅ STEP 3: Helper functions
def load_img(path_to_img):
    max_dim = 512
    img = PIL.Image.open(path_to_img)
    long = max(img.size)
    scale = max_dim / long
    img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), PIL.Image.ANTIALIAS)
    img = kp_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return tf.image.convert_image_dtype(img, tf.float32)

def deprocess_img(processed_img):
    x = processed_img.copy()
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x = np.clip(x, 0, 1)
    return x

def show_img(img, title=None):
    plt.imshow(deprocess_img(img))
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# ✅ STEP 4: Download content and style images
!wget https://upload.wikimedia.org/wikipedia/commons/0/0c/GoldenGateBridge-001.jpg -O content.jpg
!wget https://upload.wikimedia.org/wikipedia/commons/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg -O style.jpg

content_path = "content.jpg"
style_path = "style.jpg"

content_image = load_img(content_path)
style_image = load_img(style_path)

show_img(content_image, "Content Image")
show_img(style_image, "Style Image")

In [ ]:
# ✅ STEP 5: Define VGG19 layers for style/content
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    style_outputs = [vgg.get_layer(name).output for name in style_layers]
    content_outputs = [vgg.get_layer(name).output for name in content_layers]
    model_outputs = style_outputs + content_outputs
    return tf.keras.Model(vgg.input, model_outputs)

In [ ]:
# ✅ STEP 6: Feature extraction & loss
def get_feature_representations(model, content_img, style_img):
    content_img = content_img * 255.0
    style_img = style_img * 255.0

    preprocessed_content = vgg19.preprocess_input(content_img)
    preprocessed_style = vgg19.preprocess_input(style_img)

    style_outputs = model(preprocessed_style)
    content_outputs = model(preprocessed_content)

    style_features = [style_layer for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer for content_layer in content_outputs[num_style_layers:]]

    return style_features, content_features

def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    model_outputs = model(init_image)

    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]

    style_score = 0
    content_score = 0

    weight_style, weight_content = loss_weights

    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += tf.reduce_mean((gram_matrix(comb_style) - target_style) ** 2)

    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += tf.reduce_mean((comb_content - target_content) ** 2)

    style_score *= weight_style / num_style_layers
    content_score *= weight_content / num_content_layers

    loss = style_score + content_score
    return loss

@tf.function()
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    total_loss = all_loss
    return tape.gradient(total_loss, cfg['init_image']), total_loss

In [ ]:
# ✅ STEP 7: Run style transfer
def run_style_transfer(content_path, style_path, num_iterations=250, content_weight=1e3, style_weight=1e-2):
    model = get_model()
    for layer in model.layers:
        layer.trainable = False

    content_image = load_img(content_path)
    style_image = load_img(style_path)

    style_features, content_features = get_feature_representations(model, content_image, style_image)
    gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

    init_image = tf.Variable(content_image, dtype=tf.float32)
    opt = tf.optimizers.Adam(learning_rate=5.0)

    best_loss, best_img = float('inf'), None

    loss_weights = (style_weight, content_weight)
    cfg = {
        'model': model,
        'loss_weights': loss_weights,
        'init_image': init_image,
        'gram_style_features': gram_style_features,
        'content_features': content_features
    }

    for i in range(num_iterations):
        grads, loss = compute_grads(cfg)
        opt.apply_gradients([(grads, init_image)])
        clipped = tf.clip_by_value(init_image, 0.0, 1.0)
        init_image.assign(clipped)

        if loss < best_loss:
            best_loss = loss
            best_img = deprocess_img(init_image.numpy())

        if i % 50 == 0:
            print(f"Iteration {i}, Loss: {loss}")

    return best_img

stylized_img = run_style_transfer(content_path, style_path)
plt.figure(figsize=(10,10))
plt.imshow(stylized_img)
plt.title("🎨 Stylized Image")
plt.axis('off')
plt.show()